# The Meteo module

The meteo module handles the pre-processing of the atmoshpering forcing of the model. 


- Requires:

data folder. See [README](README.md) in this folder.


In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be >= 0.5
import pyposeidon
pyposeidon.__version__

In [ ]:
import pyposeidon.meteo as pt

In [ ]:
import datetime
import numpy as np
from glob import glob

### Graphics

In [ ]:
import pyposeidon.utils.pplot # initialize matplolib accessor

In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.output(widget_location='bottom')

from hvplot import xarray

In [ ]:
#optional for dark theme
#from bokeh.themes import DARK_MINIMAL
#hv.renderer('bokeh').theme = DARK_MINIMAL

In [ ]:
#optional
#pyposeidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
#optional
#import matplotlib.pyplot as plt
#plt.style.use('dark_background')

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

## Setup

The minimum information required is the geometry's extent. In the most simple case that is a lat/lon box that defines the area of interest. Without loss of generality we select below Iceland as a test case. Feel free to modify the coordinates. 

In [ ]:
#define in a dictionary the properties of the model..
geometry={'lon_min':-25., # lat/lon window
     'lon_max':-9.,
     'lat_min':56.,
     'lat_max':74.,
    }

### Remote Data

In theory, should be enough. The default setup for the time frame attribute is the current date but this depends on the source of the atmospheric data.

If the source is undefined the process defaults to retrieving GFS data from an NOAA.

The default date stamp is *today - 1 day* unless otherwise specified. 

*Note*: This option would work only for some extents. For large areas (continental) use a locally stored file. See below.

In [ ]:
meteo = pt.meteo(**geometry)

In [ ]:
meteo.Dataset

## Local Data

Most of the time, atmospheric data will be available locally. More importantly, these data are provided in a forecast mode. 

This means that there is a sequence of files that overlap in terms of time reference as new forecasts become available. 

pyposeidon provides a way to merge these data in a number of ways.

In [ ]:
# This just a structure to create the list of paths for the actual meteo files - change appropriately 
mpaths = glob('./data/uvp*.grib')
mpaths.sort()
mpaths

In [ ]:
dic = {**geometry, 'meteo_source':mpaths, 
            'meteo_engine': 'cfgrib',
            'meteo_merge':'first', 
            'meteo_combine_by' : 'nested',
            'meteo_xr_kwargs':{'concat_dim':'step'} } # The files are forecasting [0-72] so we combine them in terms of time

### retrieve meteo forcing

In [ ]:
#get uvp
meteo = pt.meteo(**dic)

In [ ]:
meteo.Dataset

**TIP** : You can change the merging with 'meteo_merge':'last'. In this case the latter values are retained in the merge

Additional engine options include 

#### passthrough

In [ ]:
meteo_ = pt.meteo(meteo_source=meteo.Dataset, meteo_engine='passthrough')

In [ ]:
meteo_.Dataset

#### empty

In [ ]:
meteo_= pt.meteo(meteo_engine='empty')

In [ ]:
meteo_.Dataset

You can now attach an already available Dataset. This is for debugging purposes.

In [ ]:
meteo_.Dataset = meteo.Dataset

## Visualise

Since the meteo Dataset is a xarray dataset all integrated viz extentions can be used to visualize the datasets.

#### with matplotlib

In [ ]:
meteo.Dataset.msl[0,:,:].plot() # A 2D graph for one timestep

### Animation

pyposeidon provides the option to get also animation as

In [ ]:
meteo.Dataset.gplot.contourf(x='longitude', y='latitude',z='msl')

### holoviews

In [ ]:
meteo.Dataset.msl.hvplot.contourf(x='longitude',y='latitude',cmap='viridis')

### geoviews

In [ ]:
import geoviews as gv
import geoviews.feature as gf

In [ ]:
g50 = gf.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
meteo.Dataset.msl.hvplot(x='longitude',y='latitude',geo=True, cmap='viridis', width=800, height=400) * g50 * gf.borders()

## Output to file depending on the solver

In [ ]:
#generic netCDF output
meteo.Dataset.to_netcdf('./test/test.nc') # to netcdf

In [ ]:
#save to DELFT3D format
meteo.to_output(solver='d3d',rpath='./test/') # to u,v,p for d3d

In [ ]:
#save to SCHISM format
meteo.to_output(solver='schism',rpath='./test/')

Additional options include

In [ ]:
# split into more files
meteo.to_output(solver='schism',rpath='./test/', meteo_split_by='day')

In [ ]:
# Define index for meteo
meteo.to_output(solver='schism',rpath='./test/', m_index=2)